In [1]:
import json
import shutil
import os
from json import JSONDecodeError

def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

from datasets import load_dataset
def GetWscTargets():

        dataset = load_dataset('stjokerli/TextToText_wsc_seqio', split='test')

        target_map={}
        for row in dataset:
            target_map[row['idx']]=row['targets']
        return target_map

def GetMultiRCTargets(input_file):
    file1 = open(input_file, 'r')
    dataset = file1.readlines()
    
    result={}
    for row in dataset:
        
#         print(row)
        
        pidx,qidx,aidx=row.split(', ')[0].split(': ')[1].split('-')
        pidx,qidx,aidx=int(pidx),int(qidx),int(aidx)
        
        target=int((row[:-2].split(', ')[1].split(': ')[1][1:-1])=='True')
        question_lablel_dict={"idx": aidx, "label": target}
        if pidx not in result.keys():
            result[pidx]={qidx:[question_lablel_dict]}
        else:
            if qidx not in result[pidx].keys():
                result[pidx][qidx]=[question_lablel_dict]
            else:
                result[pidx][qidx].append(question_lablel_dict)

    result2=[]
    for p,v in result.items():
        question_list=[]
        for q,a in v.items():
            question_list.append({"idx": q, "answers":a})
        result2.append({"idx": p, "passage":{"questions":question_list}})    

    return result2


def GetRecordTargets(input_file):
    
    file1 = open(input_file, 'r')
    dataset = file1.readlines()
    
    result=[]
    for row in dataset:
        try:
            result.append(json.loads(row[:-1]))
        except JSONDecodeError:
            temp=row[:-1].split(': "')
            temp[-1]=temp[-1][:-2].replace('"',r"'")+'"}'
            result.append(json.loads(': "'.join(temp)))
            
    return result

def MoveFiles(input_file,output_file):

    
    print('Starting',task_name)
    if task_name=='MultiRC':
        
        contents=GetMultiRCTargets(input_file)
    
    elif task_name=='ReCoRD':
        
        contents=GetRecordTargets(input_file)
        
    else:
        
        contents=load_jsonl(input_file)
                  
    if task_name in ['BoolQ','WiC']:
        for e in contents:
            e['label']=e['label'].lower()
    
    elif task_name=='COPA':
        for e in contents:
            e['label']=int(e['label'][-1])-1
    
    elif task_name=='WSC':
        target_map=GetWscTargets()
        for e in contents:
            answer=target_map[e['idx']]==e['label']
            e['label']=str(answer)
    return dump_jsonl(contents, output_file, append=False)

In [2]:
    from os import listdir
    from os.path import isfile, join
#     mypath='./Official_experiment_prediction/Mam_bn200_MNLI/'+'target_transfer_learnings'


In [3]:
# original_file_name='generated_predictions_for_submission.txt'
# model_checkpoints_root='/workspace/w266_final_project/model_checkpoints/'

In [4]:
submission_file_dict={
    "Prefix_Benchmark":["Prefix/bench_marks",
                       "Prefix_Benchmark/"],
    "Prefix_MNLI":["Prefix/Transfer_learnings",
                  "Prefix_MNLI/"],
    "Prefix_SQUAD":["Squad_Prefix/target_transfer_learnings",
                    "Prefix_SQUAD/"],
    "Prompt_Benchmark":["Prompt/bench_marks",
                       "Prompt_Benchmark/"],
    "Prompt_MNLI":["Prompt/transfer_learning",
                  "Prompt_MNLI/"],
    "Prompt_SQUAD":["Squad_Prompt/target_transfer_learnings",
                    "Prompt_SQUAD/"],
    "MAM_BN30_Benchmark":["Mam/bench_marks",
                       "MAM_BN30_Benchmark/"],
    "MAM_BN30_MNLI":["Mam/Transfer_learnings",
                  "MAM_BN30_MNLI/"],
    "MAM_BN30_SQUAD":["Squad_Mam/target_transfer_learnings",
                    "MAM_BN30_SQUAD/"],
    "MAM_BN200_Benchmark":["Mam_bn200_BENCHMARK/target_transfer_learnings/",
                          "MAM_BN200_Benchmark/"
                          ],
    "MAM_BN200_MNLI":["Mam_bn200_MNLI/target_transfer_learnings/",
                     "MAM_BN200_MNLI/"],
    "MAM_BN200_SQUAD":["Mam_bn200_SQUAD/target_transfer_learnings/",
                        "MAM_BN200_SQUAD/"]
}    

In [5]:

for key,value in submission_file_dict.items():
    
    
    mypath='./Official_experiment_prediction/'+value[0]
    output_path='/workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/'+value[1]
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    
    location_mapping={
        # they need all data inorder to submit, so we give them the dummy data
        'AX-g':'',
        'AX-b':'',


        #transfer set 10k 
        'BoolQ':'',
        'CB':'',
        'COPA':'',
        'MultiRC':'',
        'ReCoRD':'',
        'RTE':'',
        'WiC':'',
        'WSC':'',

    }
    

    for e in onlyfiles:
        taskname=e.split("_")[-1].split(".")[0].split("-")[0]
        for key in location_mapping:
            if key.lower().replace('-','')==taskname.lower():
                location_mapping[key]=mypath+'/'+e
                
                
    for _,value in location_mapping.items():
        assert value != '', f"{location_mapping}"
        
        
    for task_name,check_point_folder in location_mapping.items():

        input_file=check_point_folder
        output_file=output_path+task_name+'.jsonl'

        if os.path.isdir(output_path):
            pass
        else:
            os.mkdir(output_path)

        MoveFiles(input_file,output_file)
        
    #zip the folder
    file=shutil.make_archive(output_path, 'zip', output_path)
    # move the zip file into the folder
    shutil.move(file,output_path)

Starting AX-g
Loaded 356 records from ./Official_experiment_prediction/Prefix/bench_marks/Prefix_Benchmark_axg.txt
Wrote 356 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_Benchmark/AX-g.jsonl
Starting AX-b
Loaded 1104 records from ./Official_experiment_prediction/Prefix/bench_marks/Prefix_Benchmark_axb.txt
Wrote 1104 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_Benchmark/AX-b.jsonl
Starting BoolQ
Loaded 3245 records from ./Official_experiment_prediction/Prefix/bench_marks/Prefix_bench_marks_boolq.txt
Wrote 3245 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_Benchmark/BoolQ.jsonl
Starting CB
Loaded 250 records from ./Official_experiment_prediction/Prefix/bench_marks/generated_predictions_for_submission_prefix_benchmark_cb-AT.txt
Wrote 250 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_subm

Using custom data configuration stjokerli--TextToText_wsc_seqio-e801d03970369d66
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/stjokerli--TextToText_wsc_seqio-e801d03970369d66/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Wrote 146 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_Benchmark/WSC.jsonl
Starting AX-g
Loaded 356 records from ./Official_experiment_prediction/Prefix/Transfer_learnings/Prefix_MNLI_axg.txt
Wrote 356 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_MNLI/AX-g.jsonl
Starting AX-b
Loaded 1104 records from ./Official_experiment_prediction/Prefix/Transfer_learnings/Prefix_MNLI_axb.txt
Wrote 1104 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_MNLI/AX-b.jsonl
Starting BoolQ
Loaded 3245 records from ./Official_experiment_prediction/Prefix/Transfer_learnings/Prefix_transfer_learnings_Boolq.txt
Wrote 3245 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_MNLI/BoolQ.jsonl
Starting CB
Loaded 250 records from ./Official_experiment_prediction/Prefix/Transfer_learnings/Prefix_target_transf

Using custom data configuration stjokerli--TextToText_wsc_seqio-e801d03970369d66
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/stjokerli--TextToText_wsc_seqio-e801d03970369d66/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Wrote 146 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_MNLI/WSC.jsonl
Starting AX-g
Loaded 356 records from ./Official_experiment_prediction/Squad_Prefix/target_transfer_learnings/Prefix_SQUAD_axg.txt
Wrote 356 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_SQUAD/AX-g.jsonl
Starting AX-b
Loaded 1104 records from ./Official_experiment_prediction/Squad_Prefix/target_transfer_learnings/Prefix_SQUAD_axb.txt
Wrote 1104 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_SQUAD/AX-b.jsonl
Starting BoolQ
Loaded 3245 records from ./Official_experiment_prediction/Squad_Prefix/target_transfer_learnings/generated_predictions_for_submission_Squad_Prefix_target_transfer_learnings_boolq.txt
Wrote 3245 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/Prefix_SQUAD/BoolQ.jsonl
Starting CB
Loaded 250 rec

...

Wrote 146 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_Benchmark/WSC.jsonl
Starting AX-g
Loaded 356 records from ./Official_experiment_prediction/Mam_bn200_MNLI/target_transfer_learnings//MAM_BN200_MNLI_axg.txt
Wrote 356 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_MNLI/AX-g.jsonl
Starting AX-b
Loaded 1104 records from ./Official_experiment_prediction/Mam_bn200_MNLI/target_transfer_learnings//MAM_BN200_MNLI_axb.txt
Wrote 1104 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_MNLI/AX-b.jsonl
Starting BoolQ
Loaded 3245 records from ./Official_experiment_prediction/Mam_bn200_MNLI/target_transfer_learnings//Mnli_Mam_target_transfer_learnings_boolq.txt
Wrote 3245 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_MNLI/BoolQ.jsonl
Starting CB
Loaded 250 records from ./Of

Using custom data configuration stjokerli--TextToText_wsc_seqio-e801d03970369d66
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/stjokerli--TextToText_wsc_seqio-e801d03970369d66/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Wrote 146 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_MNLI/WSC.jsonl
Starting AX-g
Loaded 356 records from ./Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//MAM_BN200_SQUAD_axg.txt
Wrote 356 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_SQUAD/AX-g.jsonl
Starting AX-b
Loaded 1104 records from ./Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//MAM_BN200_SQUAD_axb.txt
Wrote 1104 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_SQUAD/AX-b.jsonl
Starting BoolQ
Loaded 3245 records from ./Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//Squad_Mam_bn200_transfer_learning_boolq.txt
Wrote 3245 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_SQUAD/BoolQ.jsonl
Starting CB
Loaded 250 records from ./

Using custom data configuration stjokerli--TextToText_wsc_seqio-e801d03970369d66
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/stjokerli--TextToText_wsc_seqio-e801d03970369d66/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Wrote 146 records to /workspace/w266_final_project/submissions/SuperGlue/Official_experiment_submission/MAM_BN200_SQUAD/WSC.jsonl


In [6]:
key

'WSC'

In [7]:


location_mapping

{'AX-g': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//MAM_BN200_SQUAD_axg.txt',
 'AX-b': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//MAM_BN200_SQUAD_axb.txt',
 'BoolQ': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//Squad_Mam_bn200_transfer_learning_boolq.txt',
 'CB': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//Squad_Mam_target_transfer_learnings_cb.txt',
 'COPA': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//Squad_Mam_target_transfer_learnings_copa.txt',
 'MultiRC': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//Squad_Mam_bn200_transfer_learning_multirc.txt',
 'ReCoRD': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//Squad_Mam_bn200_transfer_learning_record.txt',
 'RTE': './Official_experiment_prediction/Mam_bn200_SQUAD/target_transfer_learnings//Squad_Mam_bn200_transfer